In [2]:
from fastapi.testclient import TestClient

from service.main import app

# little hacking 
import nest_asyncio
nest_asyncio.apply()

# Przewidywanie konkretnym modelem

In [3]:
responses = []
with TestClient(app) as client:
    query = {
        "category": "gry i konsole",
        "city": "Warszawa",
        "price": 410,
        "purchase": "2021-06-01T18:37:46.571Z",
    }

    responses.append(client.post("/predict", params={"model_name": "simple" }, json=query))
    responses.append(client.post("/predict", params={"model_name": "xgboost" }, json=query))
    responses.append(client.post("/predict", params={"model_name": "random_forest" }, json=query))


In [4]:
[response.json() for response in responses]

[{'360': 37.90077141190736,
  '516': 37.90077141190736,
  '620': 37.90077141190736},
 {'360': 38.55854415893555,
  '516': 26.698657989501953,
  '620': 38.70853042602539},
 {'360': 37.119528319299995,
  '516': 28.881836373002788,
  '620': 38.86367675349167}]

# Testy AB

## Przykładowe zapytanie

Chcąc dostać przwidywany czas potrzebny na dostawe produktu request musi posiadać:
- query_id (pram) - numer identyfikujący zapytanie. Służy do ułatwienia łaczenia przewidywanch czasów z prawdziwymi
- query (body) - informacje o przesyłce

In [2]:
response = None 
with TestClient(app) as client:
    query = {
        "category": "gry i konsole",
        "city": "Warszawa",
        "price": 410,
        "purchase": "2021-06-01T18:37:46.571Z",
    }

    response = client.post("/ab-test", params={"query_id": 11 }, json=query)


query_id=11 query=Query(category='gry i konsole', city='Warszawa', price=410.0, purchase=datetime.datetime(2021, 6, 1, 18, 37, 46, 571000, tzinfo=datetime.timezone.utc)) model_name='random_forest' result={'360': 37.119528319299995, '516': 28, '620': '38.86367675349167'}


U góry znajdują się logi serwera, są one zapisywane do bazy danych.
Logowane informacje:
- query_id
- query
- model_name
- result

In [3]:
response.json()

{'360': 37.119528319299995,
 '516': 28.881836373002788,
 '620': 38.86367675349167}

## Stałe dane

In [4]:
responses = []

for i in range(6):
    with TestClient(app) as client:
        query = {
            "category": "gry i konsole",
            "city": "Warszawa",
            "price": 120,
            "purchase": "2021-06-06T18:37:46.571Z",
        }

        response = client.post("/ab-test", params={"query_id": 100 + i }, json=query)
        responses.append(response)


query_id=100 query=Query(category='gry i konsole', city='Warszawa', price=120.0, purchase=datetime.datetime(2021, 6, 6, 18, 37, 46, 571000, tzinfo=datetime.timezone.utc)) model_name='xgboost' result={'360': 40.729949951171875, '516': 29, '620': '40.76724624633789'}
query_id=101 query=Query(category='gry i konsole', city='Warszawa', price=120.0, purchase=datetime.datetime(2021, 6, 6, 18, 37, 46, 571000, tzinfo=datetime.timezone.utc)) model_name='random_forest' result={'360': 41.09714834505557, '516': 29, '620': '41.276245213361115'}
query_id=102 query=Query(category='gry i konsole', city='Warszawa', price=120.0, purchase=datetime.datetime(2021, 6, 6, 18, 37, 46, 571000, tzinfo=datetime.timezone.utc)) model_name='simple' result={'360': 37.90077141190736, '516': 37, '620': '37.90077141190736'}
query_id=103 query=Query(category='gry i konsole', city='Warszawa', price=120.0, purchase=datetime.datetime(2021, 6, 6, 18, 37, 46, 571000, tzinfo=datetime.timezone.utc)) model_name='xgboost' result

In [5]:
[response.json() for response in responses]

[{'360': 40.729949951171875,
  '516': 29.119882583618164,
  '620': 40.76724624633789},
 {'360': 41.09714834505557,
  '516': 29.063356214347227,
  '620': 41.276245213361115},
 {'360': 37.90077141190736,
  '516': 37.90077141190736,
  '620': 37.90077141190736},
 {'360': 40.729949951171875,
  '516': 29.119882583618164,
  '620': 40.76724624633789},
 {'360': 40.729949951171875,
  '516': 29.119882583618164,
  '620': 40.76724624633789},
 {'360': 41.09714834505557,
  '516': 29.063356214347227,
  '620': 41.276245213361115}]

## Zmienne dane

In [6]:
responses = []


queries = [
    {
        "category": "gry i konsole",
        "city": "Warszawa",
        "price": 120,
        "purchase": "2021-06-30T23:31:11.372Z",
    },
    {
        "category": "komputery",
        "city": "Miasto które nie istnieje",
        "price": 310,
        "purchase": "2021-04-06T16:17:47.222Z",
    },
    {
        "category": "gry i konsole",
        "city": "Mielec",
        "price": 12.2,
        "purchase": "2021-01-06T13:37:45.591Z",
    },
    {
        "category": "telefony i akcesoria",
        "city": "Inne miasto które nie istnieje",
        "price": 67.1,
        "purchase": "2021-06-12T18:27:41.544Z",
    },
    {
        "category": "telefony i akcesoria",
        "city": "Szczecin",
        "price": 21.99,
        "purchase": "2022-12-16T11:57:42.171Z",
    },
    {
        "category": "sprzęt rtv",
        "city": "Kutno",
        "price": 1300,
        "purchase": "2021-02-19T07:11:26.410Z",
    },
]


for i in range(6):
    with TestClient(app) as client:
        response = client.post("/ab-test", params={"query_id": 1000 + i}, json=queries[i])
        responses.append(response)


query_id=1000 query=Query(category='gry i konsole', city='Warszawa', price=120.0, purchase=datetime.datetime(2021, 6, 30, 23, 31, 11, 372000, tzinfo=datetime.timezone.utc)) model_name='xgboost' result={'360': 44.2491455078125, '516': 31, '620': '43.539310455322266'}
query_id=1001 query=Query(category='komputery', city='Miasto które nie istnieje', price=310.0, purchase=datetime.datetime(2021, 4, 6, 16, 17, 47, 222000, tzinfo=datetime.timezone.utc)) model_name='random_forest' result={'360': 44.20131829255278, '516': 28, '620': '43.53515402244445'}
query_id=1002 query=Query(category='gry i konsole', city='Mielec', price=12.2, purchase=datetime.datetime(2021, 1, 6, 13, 37, 45, 591000, tzinfo=datetime.timezone.utc)) model_name='xgboost' result={'360': 76.38341522216797, '516': 73, '620': '74.72496795654297'}
query_id=1003 query=Query(category='telefony i akcesoria', city='Inne miasto które nie istnieje', price=67.1, purchase=datetime.datetime(2021, 6, 12, 18, 27, 41, 544000, tzinfo=datetime

In [7]:
[response.json() for response in responses]

[{'360': 44.2491455078125,
  '516': 31.26377296447754,
  '620': 43.539310455322266},
 {'360': 44.20131829255278, '516': 28.9254894624, '620': 43.53515402244445},
 {'360': 76.38341522216797,
  '516': 73.75520324707031,
  '620': 74.72496795654297},
 {'360': 45.84796270423969,
  '516': 45.84796270423969,
  '620': 45.84796270423969},
 {'360': 36.198734283447266,
  '516': 32.825496673583984,
  '620': 35.04318618774414},
 {'360': 54.22967218170558,
  '516': 41.104450130099984,
  '620': 52.67066532346114}]